In [1]:
import pandas as pd
import gc
import numpy as np
import glob
import session_info

In [2]:
gc.collect()

15

In [3]:
import matplotlib.pyplot as plt 
import seaborn as sns 
%matplotlib inline 

# Read the data

In [4]:
tables = pd.Series( glob.glob('./alignments/*.coverage.withoutHISTO.csv') ) 
tax_data = pd.Series( glob.glob('./alignments/*.taxonomy.csv') ) 
proj_names = [ x.split('\\')[1].split('.')[0] for x in tables ]

In [5]:
tables[ tables.str.contains( proj_names[0] ) ][0], \
tax_data[ tax_data.str.contains( proj_names[0] ) ][0]

('./alignments\\PRJNA171749.coverage.withoutHISTO.csv',
 './alignments\\PRJNA171749.taxonomy.csv')

In [6]:
tab1 = pd.read_csv( tables[ tables.str.contains( proj_names[0] ) ][0] , sep = '\t')

In [7]:
tab1.head()

,#rname,startpos,endpos,numreads,covbases,coverage,meandepth,meanbaseq,meanmapq
0,MH341446:MH341446;,1,195979,3,101,0.051536,0.000515,32.0,0.0000
1,MK910851:MK910851;,1,201609,4,40,0.019840,0.000198,24.3,0.0000
2,KX689266:KX689266;,1,224384,251594,99,0.044121,79.806300,38.1,0.0564
3,MF468142:MF468142;,1,228431,1,101,0.044215,0.000442,38.1,0.0000
4,MF468147:MF468147;,1,233624,1,100,0.042804,0.000428,37.8,0.0000


In [8]:
tab1['#rname'] = tab1['#rname'].str.split(':', expand = False)
tab1 = tab1.explode(column = '#rname', ignore_index = True)
tab1['accession'] = tab1['#rname'].str.replace('[; ]','', regex = True).str.replace('\.\d+$', '', regex = True)
tab1['#rname'] = tab1['#rname'].str.replace('[; ]','', regex = True)
tab1 = tab1.drop_duplicates()

In [9]:
tab1.head()

,#rname,startpos,endpos,numreads,covbases,coverage,meandepth,meanbaseq,meanmapq,accession
0,MH341446,1,195979,3,101,0.051536,0.000515,32.0,0.0000,MH341446
2,MK910851,1,201609,4,40,0.019840,0.000198,24.3,0.0000,MK910851
4,KX689266,1,224384,251594,99,0.044121,79.806300,38.1,0.0564,KX689266
6,MF468142,1,228431,1,101,0.044215,0.000442,38.1,0.0000,MF468142
8,MF468147,1,233624,1,100,0.042804,0.000428,37.8,0.0000,MF468147


In [10]:
tab1[ tab1.numreads > 0 ]

,#rname,startpos,endpos,numreads,covbases,coverage,meandepth,meanbaseq,meanmapq,accession
0,MH341446,1,195979,3,101,0.051536,0.000515,32.0,0.0000,MH341446
2,MK910851,1,201609,4,40,0.019840,0.000198,24.3,0.0000,MK910851
4,KX689266,1,224384,251594,99,0.044121,79.806300,38.1,0.0564,KX689266
6,MF468142,1,228431,1,101,0.044215,0.000442,38.1,0.0000,MF468142
8,MF468147,1,233624,1,100,0.042804,0.000428,37.8,0.0000,MF468147
...,...,...,...,...,...,...,...,...,...,...
175,NC_048109.1,1,37152,2,190,0.511413,0.005329,32.3,4.5000,NC_048109
176,NC_049451.1,1,34141,1,101,0.295832,0.002958,29.7,7.0000,NC_049451
177,NC_049950.1,1,46288,8,748,1.615970,0.016181,34.4,14.1000,NC_049950
178,NC_049955.1,1,45560,7071,68,0.149254,0.001493,29.2,0.9600,NC_049955


In [11]:
## read the taxonomy
tax_tab1 = pd.read_csv( tax_data[ tax_data.str.contains( proj_names[0] ) ][0] , sep = '\t')

In [12]:
tab1 = tab1[ tab1.numreads > 0 ]

In [13]:
tax_tab1.sample(5)

,accession,taxid,kindom,phylum,class,order,family,genus,species
117,NC_048109,2483611,Viruses,Uroviricota,Caudoviricetes,Caudovirales,Myoviridae,Citexvirus,Citexvirus dobby
23,JF719734,10863,Viruses,Hofneiviricota,Faserviricetes,Tubulavirales,Inoviridae,Inovirus,Inovirus M13
104,NC_019711,1147148,Viruses,Uroviricota,Caudoviricetes,Caudovirales,Siphoviridae,Lambdavirus,Lambdavirus HK629
8,AY120890,1891767,Viruses,Cossaviricota,Papovaviricetes,Sepolyvirales,Polyomaviridae,Betapolyomavirus,Macaca mulatta polyomavirus 1
120,NC_049950,2847058,Viruses,Uroviricota,Caudoviricetes,Caudovirales,Siphoviridae,Glaedevirus,Glaedevirus gv2H10


In [14]:
tab1.merge( tax_tab1, how = 'left').iloc[ (tab1.covbases * tab1.coverage * tab1.meandepth).argsort()[::-1] ].head(60)

,#rname,startpos,endpos,numreads,covbases,coverage,meandepth,meanbaseq,meanmapq,accession,taxid,kindom,phylum,class,order,family,genus,species
103,NC_001422.1,1,5386,668063,5386,100.000000,11393.700000,32.8,41.700000,NC_001422,2886930.0,Viruses,Phixviricota,Malgrandaviricetes,Petitvirales,Microviridae,Sinsheimervirus,Sinsheimervirus phiX174
82,LT837585,1,12295,143989,99,0.805205,751.269000,27.4,0.000611,LT837585,11099.0,Viruses,Kitrinoviricota,Flasuviricetes,Amarillovirales,Flaviviridae,Pestivirus,Pestivirus A
19,LT837585,1,12295,143727,98,0.797072,749.518000,27.4,0.000842,LT837585,11099.0,Viruses,Kitrinoviricota,Flasuviricetes,Amarillovirales,Flaviviridae,Pestivirus,Pestivirus A
39,Z48163,1,14529,62,1838,12.650600,0.402987,34.5,5.600000,Z48163,11033.0,Viruses,Kitrinoviricota,Alsuviricetes,Martellivirales,Togaviridae,Alphavirus,Semliki Forest virus
33,KM972720,1,4467,505,121,2.708750,10.939600,34.6,0.053500,KM972720,35319.0,Viruses,Negarnaviricota,Ellioviricetes,Bunyavirales,Peribunyaviridae,Orthobunyavirus,Tete orthobunyavirus
91,KM972720,1,4467,470,119,2.663980,10.120700,34.4,0.066000,KM972720,35319.0,Viruses,Negarnaviricota,Ellioviricetes,Bunyavirales,Peribunyaviridae,Orthobunyavirus,Tete orthobunyavirus
31,MF289414,1,12310,1213,178,1.445980,8.519250,33.7,9.210000,MF289414,1980519.0,Viruses,Negarnaviricota,Ellioviricetes,Bunyavirales,Nairoviridae,Orthonairovirus,Crimean-Congo hemorrhagic fever orthonairovirus
90,KU746283,1,8109,285,181,2.232090,3.133930,34.5,0.003510,KU746283,45219.0,Viruses,Negarnaviricota,Ellioviricetes,Bunyavirales,Arenaviridae,Mammarenavirus,Guanarito mammarenavirus
22,KU746283,1,8109,342,173,2.133430,3.112840,34.5,2.490000,KU746283,45219.0,Viruses,Negarnaviricota,Ellioviricetes,Bunyavirales,Arenaviridae,Mammarenavirus,Guanarito mammarenavirus
32,KJ716849,1,4507,119,102,2.263150,1.537830,30.7,11.200000,KJ716849,273357.0,Viruses,Negarnaviricota,Ellioviricetes,Bunyavirales,Peribunyaviridae,Orthobunyavirus,Bunyamwera orthobunyavirus


In [20]:
def read_proj(proj_id):
    tab1 = pd.read_csv( tables[ tables.str.contains( proj_id ) ].values[0] , sep = '\t')
    tab1 = tab1[ tab1.numreads > 0 ]
    
    tab1['#rname'] = tab1['#rname'].str.split(':', expand = False)
    tab1 = tab1.explode(column = '#rname', ignore_index = True)
    tab1['accession'] = tab1['#rname'].str.replace(';','',regex=False).str.replace('\.\d+$', '', regex = True)
    tab1.drop(columns = '#rname', inplace = True)
    tab1 = tab1.drop_duplicates()
    
    tax_tab1 = pd.read_csv( tax_data[ tax_data.str.contains( proj_id ) ].values[0] , sep = '\t')
    
    tab2 = tab1.merge( tax_tab1, how = 'left').iloc[ 
        #(tab1.covbases * tab1.coverage * tab1.meandepth).argsort()[::-1]]
        #(tab1.covbases  * ((tab1.coverage > 15) + 1) ).argsort()[::-1]]
        #tab1.covbases .argsort()[::-1]]
        (tab1.covbases * (tab1.numreads > 10).add(1) ).argsort()[::-1]]
    
    return tab2.drop_duplicates()

In [20]:
#tables[ tables.str.contains( proj_names[1] ) ].values[0]

In [80]:
#projects_summary = pd.read_clipboard(index_col = 0)
#projects_summary.to_csv('./projects_summary.csv')

In [16]:
projects_summary = pd.read_csv('./projects_summary.csv', index_col =0 )

In [17]:
projects_summary.head()

,Instrument,Library Strategy,Library Source,Library Selection,Library Layout,Tissue,"Run of spots, sum","Run of Bases, sum",Number of samples
Project,,,,,,,,,
PRJNA508767,Illumina HiSeq 2000,OTHER,Genomic,Restriction Digest,Paired,thoracic muscle,"617,860,178","139,747,869,356",3
PRJNA369863,Illumina HiSeq 2000,OTHER,Genomic,Restriction Digest,Paired,thoracic muscle,"743,037,720","150,093,619,440",2
PRJNA171749,Illumina HiSeq 2000,WGS,Genomic,Random,Paired,Whole adult insect,"318,053,092","64,246,724,584",4
PRJNA280017,Illumina HiSeq 2000,RNA-Seq,Transcriptomic,PolyA,Paired,antenna,"51,895,266","9,341,147,880",2
PRJNA400685,Illumina HiSeq 2000,RNA-Seq,Transcriptomic,Random,Single,head,"39,831,011","2,987,325,825",9


In [18]:
nam  = proj_names[4]
tab2 = read_proj(nam)
tab2 = tab2[( tab2.covbases > 150 ) & (~ tab2['phylum'].isin(['Uroviricota','Phixviricota']) ) & \
            (~ tab2.species.fillna('').astype(str).str.contains('phage', case = False))]
idx  = tab2.groupby(['species'])['coverage'].transform(max) == tab2['coverage']
#print (nam);
#print ( projects_summary.loc[nam.split('_')[0] ] );
tab2.loc[idx].head(50);

<ipython-input-15-5d277b445e3a>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  tab1['accession'] = tab1['#rname'].str.replace(';','').str.replace('\.\d+$', '')


In [21]:
tmp_tab = pd.DataFrame()
for pn in proj_names:
    tb = read_proj(pn)
    #tb = tb[(~ tb['phylum'].isin(['Uroviricota','Phixviricota']) ) & \
    #        (~ tb.species.fillna('').astype(str).str.contains('phage', case = False))]
    idx  = tb.groupby(['species'])['coverage'].transform(max) == tb['coverage']
    tb = tb.loc[idx]
    tb['project'] = pn
    tmp_tab = pd.concat([tb, tmp_tab], axis = 0)

In [22]:
## filter out unneeded taxa:
wiki_phages = pd.read_html('https://en.wikipedia.org/wiki/Bacteriophage')[0]

In [23]:
bad_fam = tmp_tab.family.str.contains('|'.join( 
    wiki_phages.Family.str.replace('Picobirnaviridae (proposal)',
                                   'Picobirnaviridae', 
                                   regex = False)[:-1].unique() ))
bad_spec = tmp_tab.species.str.contains('(\Wphage\W)|( phage$)',
                                        case = False, regex = True)

<ipython-input-23-71ddd3cc517b>:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  bad_spec = tmp_tab.species.str.contains('(\Wphage\W)|( phage$)',


In [24]:
tmp_tab = tmp_tab[~(bad_fam | bad_spec)]

In [25]:
tmp_tab = tmp_tab.reset_index(drop = True)

In [26]:
tmp_tab.shape

(536, 18)

In [27]:
tmp_tab.head(60)

,startpos,endpos,numreads,covbases,coverage,meandepth,meanbaseq,meanmapq,accession,taxid,kindom,phylum,class,order,family,genus,species,project
0,1,8485,221,3863,45.527400,2.357930,37.3,1.000,MN950793,12169.0,Viruses,Kitrinoviricota,Alsuviricetes,Tymovirales,Betaflexiviridae,Carlavirus,Potato virus S,PRJNA694179
1,1,9708,1,91,0.937371,0.009374,38.9,6.000,KT336551,12216.0,Viruses,Pisuviricota,Stelpaviricetes,Patatavirales,Potyviridae,Potyvirus,Potato virus Y,PRJNA694179
2,1,104710,4,67,0.063986,0.000945,31.3,0.750,DQ333351,56947.0,Viruses,unclassified Viruses phylum,Naldaviricetes,Lefavirales,Baculoviridae,Betabaculovirus,Choristoneura fumiferana granulovirus,PRJNA694179
3,1,9694,21708,9561,98.628000,224.066000,36.5,4.470,EF026075,12216.0,Viruses,Pisuviricota,Stelpaviricetes,Patatavirales,Potyviridae,Potyvirus,Potato virus Y,PRJNA646009
4,1,8485,480252,8418,99.210400,5632.850000,35.7,1.020,MN950792,12169.0,Viruses,Kitrinoviricota,Alsuviricetes,Tymovirales,Betaflexiviridae,Carlavirus,Potato virus S,PRJNA646009
5,1,396,16,240,60.606100,3.911620,33.3,2.190,FJ751933,596768.0,Viruses,unclassified Viruses phylum,unclassified Viruses class,unclassified Viruses order,Pospiviroidae,Pospiviroid,Citrus exocortis viroid,PRJNA646009
6,1,138991,4,288,0.207208,0.002907,35.6,1.000,KU697903,46015.0,Viruses,unclassified Viruses phylum,Naldaviricetes,Lefavirales,Baculoviridae,Alphabaculovirus,Autographa californica multiple nucleopolyhedr...,PRJNA646009
7,1,3753,49,140,3.730350,1.262990,34.7,28.800,MK913782,12321.0,Viruses,Kitrinoviricota,Alsuviricetes,Martellivirales,Bromoviridae,Alfamovirus,Alfalfa mosaic virus,PRJNA646009
8,1,104710,28,108,0.103142,0.014440,28.1,0.821,NC_008168,56947.0,Viruses,unclassified Viruses phylum,Naldaviricetes,Lefavirales,Baculoviridae,Betabaculovirus,Choristoneura fumiferana granulovirus,PRJNA646009
9,1,229209,2,202,0.088129,0.000881,36.6,1.000,AC146907,10359.0,Viruses,Peploviricota,Herviviricetes,Herpesvirales,Herpesviridae,Cytomegalovirus,Human betaherpesvirus 5,PRJNA646009


In [28]:
tmp_tab.columns

Index(['startpos', 'endpos', 'numreads', 'covbases', 'coverage', 'meandepth',
       'meanbaseq', 'meanmapq', 'accession', 'taxid', 'kindom', 'phylum',
       'class', 'order', 'family', 'genus', 'species', 'project'],
      dtype='object')

In [29]:
ohe_proj = pd.get_dummies(tmp_tab.project, prefix = 'proj')

In [30]:
tmp_tab2 = pd.concat([tmp_tab, ohe_proj], axis = 1)

In [31]:
convert_dict = {'accession' : str,
                'kindom' : str,
                'phylum' : str,
                'class'  : str,
                'order'  : str,
                'family' : str,
                'genus'  : str, 
                'species': str,
                'project':str
               }

In [32]:
tmp_tab = tmp_tab.astype(convert_dict)

In [33]:
tmp_tab_family = tmp_tab.groupby( 'family' ).agg({
    'numreads' : ['min', 'max', 'mean','median'], 
    'covbases' : ['min', 'max', 'mean','median'],
    'coverage' : ['min', 'max', 'mean','median'],
    'endpos'   : ['min', 'max', 'mean','median'],
    'accession': ['nunique'],
    'project'  : ['nunique', lambda x: ';'.join(set(x))]
})
tmp_tab_family1 = tmp_tab_family.sort_values( [('project', 'nunique'), ('covbases', 'max')], ascending = [False, False] )
tmp_tab_family1 = tmp_tab_family1[(tmp_tab_family1 [('covbases', 'max')] > 250) & \
                                  (tmp_tab_family1 [('numreads', 'max')] > 4)]

In [34]:
def style_df(df):
    return df.style\
.format("{:.2f}", subset = [
            ('endpos',         'mean'),
            ('endpos',       'median'),
            ('numreads',       'mean'),
            ('numreads',     'median'),
            ('covbases',       'mean'),
            ('covbases',     'median'),
            ('coverage',        'min'),
            ('coverage',        'max'),
            ('coverage',       'mean'),
            ('coverage',     'median')] )\
    .background_gradient(
      subset = [
            ('endpos',        'min'),
            ('endpos',        'max'),
            ('endpos',       'mean'),
            ('endpos',     'median'),
            ('numreads',        'min'),
            ('numreads',        'max'),
            ('numreads',       'mean'),
            ('numreads',     'median'),
            ('covbases',        'min'),
            ('covbases',        'max'),
            ('covbases',       'mean'),
            ('covbases',     'median'),
            ('coverage',        'min'),
            ('coverage',        'max'),
            ('coverage',       'mean'),
            ('coverage',     'median')], cmap='coolwarm', axis = 0, high = .01, low = 0.01)\
     .set_table_styles([ dict(selector='td', props=[('text-align', 'left')]),
                         dict(selector='td', props=[('font-size', '8pt')] ),
                         dict(selector='th', props=[('text-align', 'left')]),
                         dict(selector='th', props=[('font-size', '8pt')] )])

In [35]:
style_df( tmp_tab_family1 )

In [36]:
tmp_tab_genus = tmp_tab.groupby( ['family', 'genus'] ).agg({
    'numreads' : ['min', 'max', 'mean','median'], 
    'covbases' : ['min', 'max', 'mean','median'],
    'coverage' : ['min', 'max', 'mean','median'],
    'endpos'   : ['min', 'max', 'mean','median'],
    'accession': ['nunique'],
    'project'  : ['nunique', lambda x: ';'.join(set(x))]
})
tmp_tab_genus_f = tmp_tab_genus[( tmp_tab_genus.loc[:, ('covbases','max')] > 250 ) & \
                                  (tmp_tab_genus.loc[:, ('numreads','max')] > 4)]
tmp_tab_genus_f1 = tmp_tab_genus_f.sort_values( [('covbases', 'max'), ('project', 'nunique')], 
                                               ascending = [False, False] )#.head(50)

In [37]:
style_df ( tmp_tab_genus_f1 )

In [38]:
tmp_tab_spec = tmp_tab.groupby( ['family', 'genus', 'species'] ).agg({
    'numreads' : ['min', 'max', 'mean','median'], 
    'covbases' : ['min', 'max', 'mean','median'],
    'coverage' : ['min', 'max', 'mean','median'],
    'endpos'   : ['min', 'max', 'mean','median'],
    'accession': ['nunique'],
    'project'  : ['nunique', lambda x: ';'.join(set(x))]
})
#tmp_tab_spec_f = tmp_tab_spec[( tmp_tab_spec.loc[:, ('covbases','max')] > 300 ) & \
#                                  (tmp_tab_spec.loc[:, ('numreads','max')] > 20) & \
#                                    (tmp_tab_spec.loc[:, ('coverage','max')] > .5) ]
#tmp_tab_spec_f1 = tmp_tab_spec_f.sort_values( [('covbases', 'max'), ('project', 'nunique')], 
#                                             ascending = [False, False] )#.head(50)
tmp_tab_spec_f1 = tmp_tab_spec.sort_values( [('covbases', 'max'), ('project', 'nunique')], 
                                             ascending = [False, False] )#.head(50)

In [39]:
style_df( tmp_tab_spec_f1 )

In [137]:
#style_df( tmp_tab_spec_f1 ).to_excel('alignment_results_unf.xlsx')

In [41]:
tmp_tab_spec_f2  = tmp_tab_spec_f1.sort_values( [('project', 'nunique'), ('covbases', 'max')],
                                              ascending = [False, False] )#.head(50)

In [42]:
style_df( tmp_tab_spec_f2 )

In [46]:
projects_summary.loc['PRJNA280017']

Instrument           Illumina HiSeq 2000
Library Strategy                 RNA-Seq
Library Source            Transcriptomic
Library Selection                  PolyA
Library Layout                    Paired
Tissue                           antenna
Run of spots, sum             51,895,266
Run of Bases, sum          9,341,147,880
Number of samples                      2
Name: PRJNA280017, dtype: object

In [47]:
projects_summary.loc['PRJNA275431']

Instrument           Illumina HiSeq 2500
Library Strategy                 RNA-Seq
Library Source            Transcriptomic
Library Selection                 RANDOM
Library Layout                    Paired
Tissue                       Whole larva
Run of spots, sum            145,371,374
Run of Bases, sum         29,365,017,548
Number of samples                     12
Name: PRJNA275431, dtype: object

In [43]:
tmp_tab_spec_f = tmp_tab_spec[( tmp_tab_spec.loc[:, ('covbases','max')] > 250 ) & \
                                  (tmp_tab_spec.loc[:, ('numreads','max')] > 4)]
tmp_tab_spec_f3 = tmp_tab_spec_f.sort_values( [('coverage', 'max'), ('project', 'nunique')], 
                                             ascending = [False, False] )#.head(50)
style_df( tmp_tab_spec_f3 )

In [44]:
tmp_tab_spec_f = tmp_tab_spec[( tmp_tab_spec.loc[:, ('covbases','max')] > 250 ) & \
                                  (tmp_tab_spec.loc[:, ('numreads','max')] > 4)]
tmp_tab_spec_f4 = tmp_tab_spec_f.sort_values( [('numreads', 'max'), ('project', 'nunique')], 
                                             ascending = [False, False] )#.head(50)
style_df( tmp_tab_spec_f4 )

In [45]:
session_info.show(excludes = ['pyarrow','distributed'])